In [24]:
import json
import numpy as np
from pathlib import Path
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf

In [26]:
# Load your existing JSON data
with open('C:/Users/Ajmel/Desktop/Ai_c/try/Travel-Guidence-Chatbot/chatbot_backend/chatapi/utils/baale_mountain.json', encoding='utf-8') as file:
    intents = json.load(file)

In [29]:
# Prepare data
texts = []
labels = []
for intent in intents['intents']:
    for pattern in intent['patterns']:
        texts.append(pattern)
        labels.append(intent['tag'])

In [30]:
# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

In [31]:
# Save label encoder
np.save('bert_label_encoder.npy', label_encoder.classes_)

# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


In [32]:
# Tokenize data
tokenized_data = tokenizer(
    texts,
    padding=True,
    truncation=True,
    max_length=128,
    return_tensors='tf'
)

In [33]:
# Create TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices((
    dict(tokenized_data),
    encoded_labels
)).shuffle(1000).batch(16)

# Initialize model
model = TFBertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=len(label_encoder.classes_)
)


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
# Compile and train
model.compile(
    optimizer=tf.keras.optimizers.Adam(3e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

model.fit(dataset, epochs=10)

Epoch 1/10
4/4 [==============================] - 49s 2s/step - loss: 2.6127 - accuracy: 0.2353
Epoch 2/10
4/4 [==============================] - 10s 2s/step - loss: 2.5215 - accuracy: 0.3137
Epoch 3/10
4/4 [==============================] - 9s 2s/step - loss: 2.4221 - accuracy: 0.2941
Epoch 4/10
4/4 [==============================] - 9s 2s/step - loss: 2.3195 - accuracy: 0.4706
Epoch 5/10
4/4 [==============================] - 9s 2s/step - loss: 2.2725 - accuracy: 0.5098
Epoch 6/10
4/4 [==============================] - 9s 2s/step - loss: 2.1607 - accuracy: 0.5294
Epoch 7/10
4/4 [==============================] - 9s 2s/step - loss: 2.0900 - accuracy: 0.5686
Epoch 8/10
4/4 [==============================] - 10s 2s/step - loss: 1.9854 - accuracy: 0.6667
Epoch 9/10
4/4 [==============================] - 9s 2s/step - loss: 1.9038 - accuracy: 0.6275
Epoch 10/10
4/4 [==============================] - 9s 2s/step - loss: 1.8280 - accuracy: 0.7843


In [36]:
# Save model and tokenizer
model.save_pretrained('bert_baale_model')
tokenizer.save_pretrained('bert_baale_model')

print("✅ Training complete - Model saved to 'bert_baale_model'")

✅ Training complete - Model saved to 'bert_baale_model'
